In [2]:
import re

file = open('inputs/day17.txt').read()

# Read all integers in the string file
reg_a, reg_b, reg_c, *program = [int(x) for x in re.findall(r'-?\d+', file)]

def get_value_combo_operand(operand, reg_a, reg_b, reg_c):
    if 0 <= operand <= 3: 
        return operand # literal operand
    elif operand == 4: 
        return reg_a
    elif operand == 5:
        return reg_b
    elif operand == 6:
        return reg_c
    else:
        raise ValueError(f'Invalid operand {operand}')
    

# reg_a_vs_numbers = list()    
def run_program(program, reg_a, reg_b, reg_c, is_part_2 = False, debug=False):
    instruction_pointer = 0
    output_array = []
    while instruction_pointer < len(program): 
        opcode, operand = program[instruction_pointer], program[instruction_pointer + 1]

        if opcode == 0: # adv instruction
            # perform divisions
            numerator = reg_a
            denominator = get_value_combo_operand(operand, reg_a, reg_b, reg_c)
            denominator = 2**denominator
            reg_a = numerator // denominator
            instruction_pointer += 2

        elif opcode == 1: # bxl instruction 
            # perform bitwise XOR
            reg_b = reg_b ^ operand
            instruction_pointer += 2

        elif opcode == 2: # bst instruction
            # calculate combo operand modulo 8
            operator = get_value_combo_operand(operand, reg_a, reg_b, reg_c)
            reg_b = operator % 8
            instruction_pointer += 2
            
        elif opcode == 3: # jnz instruction
            # jump not zero
            if reg_a == 0:
                instruction_pointer += 2
            else: 
                instruction_pointer = operand

        elif opcode == 4: # bxc instruction
            # bitwise xor
            reg_b = reg_b ^ reg_c
            instruction_pointer += 2

        elif opcode == 5: # out instruction
            # output value
            output_array.append(get_value_combo_operand(operand, reg_a, reg_b, reg_c) % 8)
            if is_part_2: 
                return output_array[-1], reg_a
            instruction_pointer += 2

        elif opcode == 6: # bdv instruction
            # perform divisions
            numerator = reg_a
            denominator = get_value_combo_operand(operand, reg_a, reg_b, reg_c)
            denominator = 2**denominator
            reg_b = numerator // denominator
            instruction_pointer += 2
        elif opcode == 7: # cdv instruction
            # perform divisions
            numerator = reg_a
            denominator = get_value_combo_operand(operand, reg_a, reg_b, reg_c)
            denominator = 2**denominator
            reg_c = numerator // denominator
            instruction_pointer += 2
    if debug: 
        print(f"reg_a: {reg_a}, reg_b: {reg_b}, reg_c: {reg_c}")
    return output_array

start_reg_a = reg_a    
output_array = run_program(program, reg_a, reg_b, reg_c)
print("Part 1", ','.join(map(str, output_array)))

def stupid_computer(reg_a): 
    reg_b = reg_a % 8 # Last 3 bytes
    reg_b = reg_b ^ 3 # XOR with 3
    reg_c = reg_a // (2**reg_b)
    reg_a = reg_a // (2**3)
    reg_b = reg_b ^ 5
    reg_b = reg_b ^ reg_c
    out = reg_b % 8 # Last 3 bytes
    return out, reg_a

reg_a = start_reg_a
output_array = []

while True: 
    out, reg_a = stupid_computer(reg_a)
    output_array.append(out)
    if reg_a == 0: 
        break

print("Part 1 attempt 2", ','.join(map(str, output_array)))

# Part 2
# Note that the program halts if reg_a is 0, and that reg_a always gets divided by 8
# The output part is more difficult to determine, but we can brute force it quickly
possible_rega = list(range(8))
for length_last in range(1, len(program)+1):
    found_numbers = list()
    for reg_a in possible_rega:
        output_array = run_program(program, reg_a, 0, 0)
        if output_array == program[-length_last:]:
            found_numbers.append(reg_a)

    next_possible = list()
    for reg_a in found_numbers:
        for x in range(8): 
            next_possible.append(reg_a * 8 + x)

    possible_rega = next_possible
print("Part 2:", min(found_numbers))

Part 1 1,6,7,4,3,0,5,0,6
Part 1 attempt 2 1,6,7,4,3,0,5,0,6
Part 2: 216148338630253
